# Session 3 - ML for Classification - Homework
#### By Quynh Le

Part of the [Machine Learning Zoomcamp (September 2021 – December 2021)](https://datatalks.club/courses/2021-winter-ml-zoomcamp.html)

In [210]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv

--2021-09-24 11:56:31--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: 'AB_NYC_2019.csv'

     0K .......... .......... .......... .......... ..........  0%  362K 19s
    50K .......... .......... .......... .......... ..........  1% 12.3M 10s
   100K .......... .......... .......... .......... ..........  2% 38.2M 6s
   150K .......... .......... .......... .......... ..........  2% 1.16M 6s
   200K .......... .......... .......... .......... ..........  3% 21.4M 5s
   250K .......... .......... .......... .......... ..........  4% 11.3M 4s
   300K .......... .......... .......... .......... ..........  5% 1.27M 4s
   350K .......... ...

In [211]:
data = pd.read_csv('AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


- Columns' names are already lowercase and have no space
- Values in categorical variable need to be cleaned

In [214]:
categorical_columns = list(data.dtypes[data.dtypes == 'object'].index)

for c in categorical_columns: 
    data[c] = data[c].str.lower().str.replace(' ', '_')

In [215]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,clean_&_quiet_apt_home_by_the_park,2787,john,brooklyn,kensington,40.64749,-73.97237,private_room,149,1,9,2018-10-19,0.21,6,365
1,2595,skylit_midtown_castle,2845,jennifer,manhattan,midtown,40.75362,-73.98377,entire_home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,the_village_of_harlem....new_york_!,4632,elisabeth,manhattan,harlem,40.80902,-73.94190,private_room,150,3,0,NaN,NaN,1,365
3,3831,cozy_entire_floor_of_brownstone,4869,lisaroxanne,brooklyn,clinton_hill,40.68514,-73.95976,entire_home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,entire_apt:_spacious_studio/loft_by_central_park,7192,laura,manhattan,east_harlem,40.79851,-73.94399,entire_home/apt,80,10,9,2018-11-19,0.10,1,0


In [216]:
# Select variables to be in the data:
select = ['neighbourhood_group',
          'room_type', 
          'latitude', 
          'longitude', 
          'price',
          'minimum_nights', 
          'number_of_reviews',
          'reviews_per_month', 
          'calculated_host_listings_count',
          'availability_365']

In [217]:
data = data[select]
data.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,brooklyn,private_room,40.64749,-73.97237,149,1,9,0.21,6,365
1,manhattan,entire_home/apt,40.75362,-73.98377,225,1,45,0.38,2,355
2,manhattan,private_room,40.80902,-73.94190,150,3,0,NaN,1,365
3,brooklyn,entire_home/apt,40.68514,-73.95976,89,1,270,4.64,1,194
4,manhattan,entire_home/apt,40.79851,-73.94399,80,10,9,0.10,1,0


In [222]:
data.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

- 2 variables are categorical: 'neighbourhood_group' and 'room_type'
- The rest 8 variables are numerical

In [218]:
data.isnull().sum()

neighbourhood_group                   0
room_type                             0
latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [219]:
# Fill the missing values with 0:
data['reviews_per_month'] = data['reviews_per_month'].fillna(0)

In [220]:
data.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [224]:
# Create a new 'above_average' variable: 1 if price >= 152, else 0
data['above_average'] = (data['price'] >= 152).astype(int)
data.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,brooklyn,private_room,40.64749,-73.97237,149,1,9,0.21,6,365,0
1,manhattan,entire_home/apt,40.75362,-73.98377,225,1,45,0.38,2,355,1
2,manhattan,private_room,40.80902,-73.94190,150,3,0,0.00,1,365,0
3,brooklyn,entire_home/apt,40.68514,-73.95976,89,1,270,4.64,1,194,0
4,manhattan,entire_home/apt,40.79851,-73.94399,80,10,9,0.10,1,0,0


**Question 1:**
The most frequent observation (mode) for the column is Manhattan.

In [225]:
data.neighbourhood_group.value_counts()

manhattan        21661
brooklyn         20104
queens            5666
bronx             1091
staten_island      373
Name: neighbourhood_group, dtype: int64

### Split the data

In [226]:
from sklearn.model_selection import train_test_split

In [227]:
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)

In [228]:
len(df_full_train), len(df_test)

(39116, 9779)

In [229]:
df_train, df_val = train_test_split(df_full_train, test_size=0.2/0.8, random_state=42)

In [230]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [231]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [233]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

In [234]:
del df_train['price'], df_train['above_average']
del df_val['price'], df_val['above_average']
del df_test['price'], df_test['above_average']

In [235]:
df_train.head()

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,brooklyn,entire_home/apt,40.72760,-73.94495,3,29,0.70,13,50
1,manhattan,private_room,40.70847,-74.00498,1,0,0.00,1,7
2,bronx,entire_home/apt,40.83149,-73.92766,40,0,0.00,1,0
3,brooklyn,entire_home/apt,40.66448,-73.99407,2,3,0.08,1,0
4,manhattan,private_room,40.74118,-74.00012,1,48,1.80,2,67


In [236]:
y_train

array([0, 0, 0, ..., 1, 0, 0])

In [237]:
len(y_train), len(y_val), len(y_test)

(29337, 9779, 9779)

**Question 2:**
- Correlation matrix is provided below.
- Two features that have the biggest correlation in this dataset are 'number_of_reviews' and 'reviews_per_month'.

In [238]:
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [239]:
numerical = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']

In [240]:
df_train[numerical].corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


**Question 3:** 
Between the two categorical variables, 'room_type' has bigger score. 

In [241]:
from sklearn.metrics import mutual_info_score

In [242]:
df_train.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [243]:
categorical = ['neighbourhood_group', 'room_type']

In [244]:
# Calculate mutual information score with the binary price for two categorical variables (only training data):
def mutual_info_binary(series):
    return mutual_info_score(series, above_average)

In [245]:
mi = round(df_train[categorical].apply(mutual_info_binary), 2)
mi.sort_values(ascending=False)

room_type              0.14
neighbourhood_group    0.05
dtype: float64

**Question 4:**
The accuracy of the model on the validation set is 0.79.

In [246]:
from sklearn.feature_extraction import DictVectorizer

In [247]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts[0]

{'neighbourhood_group': 'brooklyn',
 'room_type': 'entire_home/apt',
 'latitude': 40.7276,
 'longitude': -73.94495,
 'minimum_nights': 3,
 'number_of_reviews': 29,
 'reviews_per_month': 0.7,
 'calculated_host_listings_count': 13,
 'availability_365': 50}

In [248]:
dv = DictVectorizer(sparse=False)

In [249]:
X_train = dv.fit_transform(train_dicts)

In [250]:
X_train.shape

(29337, 15)

In [251]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
val_dicts[0]

{'neighbourhood_group': 'brooklyn',
 'room_type': 'private_room',
 'latitude': 40.70239,
 'longitude': -73.92931,
 'minimum_nights': 1,
 'number_of_reviews': 35,
 'reviews_per_month': 1.8,
 'calculated_host_listings_count': 1,
 'availability_365': 52}

In [252]:
X_val = dv.transform(val_dicts)
X_val.shape

(9779, 15)

Now we have feature matrices for training and validation set ready.

In [253]:
from sklearn.linear_model import LogisticRegression

In [256]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [257]:
model.intercept_[0]

-0.002639655005405582

In [258]:
model.coef_[0]

array([ 0.00293039,  0.00394278, -0.22895717, -0.09550676, -0.01247936,
       -0.39911445,  0.08083951,  1.24633724, -0.79875565, -0.13194064,
       -0.00326298, -0.04206671,  1.63006591, -1.15864509, -0.47405481])

In [261]:
predict_val = model.predict(X_val)
predict_val

array([0, 1, 0, ..., 0, 0, 1])

In [263]:
# Accuracy of model on validation set:
(predict_val == y_val).mean().round(2)

0.79

**Question 5:**
The feature has the smallest difference is either 'number_of_reviews' or 'reviews_per_month'

In [306]:
full_model = categorical + numerical
full_model

['neighbourhood_group',
 'room_type',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [323]:
model_ng = ['room_type','latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
model_rt = ['neighbourhood_group','latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
model_lat = ['neighbourhood_group','room_type','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
model_long = ['neighbourhood_group','room_type','latitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
model_min_n = ['neighbourhood_group','room_type','latitude','longitude','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
model_num_r = ['neighbourhood_group','room_type','latitude','longitude','minimum_nights','reviews_per_month','calculated_host_listings_count','availability_365']
model_revpm = ['neighbourhood_group','room_type','latitude','longitude','minimum_nights','number_of_reviews','calculated_host_listings_count','availability_365']
model_cal_count = ['neighbourhood_group','room_type','latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','availability_365']
model_aval = ['neighbourhood_group','room_type','latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count']

In [309]:
len(model_r0),len(model_r1),len(model_r2),len(model_r3),len(model_r4),len(model_r5),len(model_r6),len(model_r7),len(model_r8)

(8, 8, 8, 8, 8, 8, 8, 8, 8)

In [312]:
def logistic_model(model):
    #Prepare feature matrix:
    train_dicts = df_train[model].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)
    val_dicts = df_val[model].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    #Train model:
    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    model.fit(X_train, y_train)
    
    #Predict using validation data:
    predict_val = model.predict(X_val)
    predict_val
    
    #Calculate accuracy on validation data:
    return (predict_val == y_val).mean().round(2)

In [333]:
(predict_val == y_val).mean().round(2) - logistic_model(model_ng)

0.040000000000000036

In [334]:
(predict_val == y_val).mean().round(2) - logistic_model(model_rt)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.07000000000000006

In [335]:
(predict_val == y_val).mean().round(2) - logistic_model(model_lat)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [336]:
(predict_val == y_val).mean().round(2) - logistic_model(model_long)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [337]:
(predict_val == y_val).mean().round(2) - logistic_model(model_min_n)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [338]:
(predict_val == y_val).mean().round(2) - logistic_model(model_num_r)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [339]:
(predict_val == y_val).mean().round(2) - logistic_model(model_revpm)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [340]:
(predict_val == y_val).mean().round(2) - logistic_model(model_cal_count)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [341]:
(predict_val == y_val).mean().round(2) - logistic_model(model_aval)

C:\Users\quynh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.010000000000000009

**Question 6:**
alpha = 0 leads to the best RMSE on the validation set.

In [360]:
from sklearn.linear_model import Ridge

In [361]:
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.2/0.8, random_state=42)

In [362]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [363]:
df_train.head()

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,brooklyn,entire_home/apt,40.72760,-73.94495,99,3,29,0.70,13,50,0
1,manhattan,private_room,40.70847,-74.00498,57,1,0,0.00,1,7,0
2,bronx,entire_home/apt,40.83149,-73.92766,70,40,0,0.00,1,0,0
3,brooklyn,entire_home/apt,40.66448,-73.99407,130,2,3,0.08,1,0,0
4,manhattan,private_room,40.74118,-74.00012,110,1,48,1.80,2,67,0


In [364]:
y_train = np.log1p(df_train.price.values)
y_val = np.log1p(df_val.price.values)
y_test = np.log1p(df_test.price.values)

In [365]:
y_train

array([4.60517019, 4.06044301, 4.26267988, ..., 5.70378247, 4.18965474,
       4.53259949])

In [366]:
len(y_train), len(y_val), len(y_test)

(29337, 9779, 9779)

In [367]:
del df_train['price'], df_train['above_average']
del df_val['price'], df_val['above_average']
del df_test['price'], df_test['above_average']

In [368]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [373]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [378]:
for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a)
    model.fit(X_train, y_train)
    
    y_pred_val = model.predict(X_val)
    
    print(rmse(y_val, y_pred_val))

0.4970735408645011
0.4971173046190632
0.49711832446943993
0.49713953633200497
0.4978866015876555


In [384]:
for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a)
    model.fit(X_train, y_train)
    
    y_pred_val = model.predict(X_val)
    
    print(round(rmse(y_val, y_pred_val),5))

0.49707
0.49712
0.49712
0.49714
0.49789


In [385]:
for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a)
    model.fit(X_train, y_train)
    
    y_pred_val = model.predict(X_val)
    
    print(round(rmse(y_val, y_pred_val),3))

0.497
0.497
0.497
0.497
0.498
